#### This notebook would be used to run the code on reduced dataset which can then offer finer controls over the parameter space.

In [1]:
cd merity

/data/priyansh/new_kranti/merity


In [2]:
import model as merity

In [3]:
cd ..

/data/priyansh/new_kranti


In [4]:
# Torch imports
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import  DataLoader
from torch.nn.utils.rnn import pad_packed_sequence, pack_sequence 

# Local imports
import data_loader as dl
import auxiliary as aux
import network as net

# Other libs
from tqdm import tqdm
import numpy as np
import argparse
import time
import sys

from configs import config_loader as cl
import corechain as cc
import components as com
import utils.tensor_utils as tu

sys.path.append('/data/priyansh/conda/fastai')
from fastai.text import *
import fastai
from fastai import text, core, lm_rnn
from typing import Any, AnyStr, Callable, Collection, Dict, Hashable, Iterator, List, Mapping, NewType, Optional
import matplotlib.pyplot as plt


Loading ULMFIT vocab and vectors from disk. Sit Tight.
Label Cache not found. Creating a new one


Traceback (most recent call last):
  File "/data/priyansh/new_kranti/utils/dbpedia_interface.py", line 133, in __init__
    self.labels = pickle.load(open('resources/labels.pickle'))
TypeError: a bytes-like object is required, not 'str'


NOT IMPLEMENTED FUNCTIONALITY NOT IMPLEMENTED FUNCTIONALITY !!!


In [5]:
#setting up device,model name and loss types.
device = torch.device("cuda")
training_model = 'bilstm_dot_ulmfit'
_dataset = 'lcquad'
_train_over_validation = False
pointwise = False
_train_over_validation = False
finetune = False
bidirectional = True

#Loading relations file.
COMMON_DATA_DIR = 'data/data/common'
_dataset_specific_data_dir = 'data/data/%(dataset)s/' % {'dataset': _dataset}
_inv_relations = aux.load_inverse_relation(COMMON_DATA_DIR)
_word_to_id = aux.load_word_list(COMMON_DATA_DIR)

NOT IMPLEMENTED FUNCTIONALITY NOT IMPLEMENTED FUNCTIONALITY !!!


In [6]:
if pointwise:
    training_config = 'pointwise'
else:
    training_config = 'pairwise'

if training_model == 'reldet':
    schema = 'reldet'
elif training_model == 'slotptr' or training_model == 'slotptr_common_encoder' or training_model == 'slotptrortho':
    schema = 'slotptr'
elif training_model == 'bilstm_dot_multiencoder':
    schema = 'default'
else:
    schema = 'default'

In [7]:
parameter_dict = cl.corechain_parameters(dataset=_dataset,training_model=training_model,
                                             training_config=training_config,config_file='configs/macros.cfg')

parameter_dict['batch_size'] = 400
if _dataset == 'lcquad':
    test_every = parameter_dict['test_every']
else:
    test_every = False
validate_every = parameter_dict['validate_every']


data = aux.load_data(_dataset=_dataset, _train_over_validation = _train_over_validation,
                     _parameter_dict=parameter_dict, _relations =  _inv_relations, _pointwise=pointwise, _device=device,k=1000)

train_loader = cc.load_data(data, parameter_dict, pointwise, schema=schema)

(700, 25)
(700, 25)
(700, 1000, 25)
(100, 25)
(100, 25)
(100, 1000, 25)
(200, 25)
(200, 25)
(200, 1000, 25)
(700, 25)


### Copy pasting the RNN core module from the fastai

In [8]:
import warnings
from fastai.imports import *
from fastai.torch_imports import *
from fastai.rnn_reg import LockedDropout,WeightDrop,EmbeddingDropout
from fastai.model import Stepper
from fastai.core import set_grad_enabled
import collections

from torch import Tensor
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pack_sequence
from typing import Sequence, Tuple, TypeVar, Union
# from .imports.core import *
# from fastai.imports.core import *
# IS_TORCH_04 = LooseVersion(torch.__version__) >= LooseVersion('0.4')

def seq2seq_reg(output, xtra, loss, alpha=0, beta=0):
    hs,dropped_hs = xtra
    if alpha:  # Activation Regularization
        loss = loss + (alpha * dropped_hs[-1].pow(2).mean()).sum()
    if beta:   # Temporal Activation Regularization (slowness)
        h = hs[-1]
        if len(h)>1: loss = loss + (beta * (h[1:] - h[:-1]).pow(2).mean()).sum()
    return loss

def repackage_var(h):
    """Wraps h in new Variables, to detach them from their history."""
    if IS_TORCH_04: return h.detach() if type(h) == torch.Tensor else tuple(repackage_var(v) for v in h)
    else: return Variable(h.data) if type(h) == Variable else tuple(repackage_var(v) for v in h)
    
# def dropout_mask(x,sz,p):
#     "Return a dropout mask of the same type as x, size sz, with probability p to cancel an element."
#     return x.new(*sz).bernoulli_(1-p).div_(1-p)

# class RNNDropout(nn.Module):
#     "Dropout that is consistent on the seq_len dimension."
#     def __init__(self, p:float=0.5):
#         super().__init__()
#         self.p=p

#     def forward(self, x:Tensor) -> Tensor:
#         if not self.training or self.p == 0.: return x
#         m = dropout_mask(x.data, (1, x.size(1), x.size(2)), self.p)
#         return x * m

# class WeightDropout(nn.Module):
#     "A module that warps another layer in which some weights will be replaced by 0 during training."

#     def __init__(self, module, weight_p, layer_names=['weight_hh_l0']):
#         super().__init__()
#         self.collection = {}
#         self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
#         for layer in self.layer_names:
#             #Makes a copy of the weights of the selected layers.
#             w = getattr(self.module, layer)
#             self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))

#     def _setweights(self):
#         "Apply dropout to the raw weights."
#         for layer in self.layer_names:
#             raw_w = getattr(self, f'{layer}_raw')
#             self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)

#     def forward(self, *args):
#         self._setweights()
#         with warnings.catch_warnings():
#             #To avoid the warning that comes because the weights aren't flattened.
#             warnings.simplefilter("ignore")
#             return self.module.forward(*args)

#     def reset(self):
#         for layer in self.layer_names:
#             raw_w = getattr(self, f'{layer}_raw')
#             self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=False)
#         if hasattr(self.module, 'reset'): self.module.reset()

# class EmbeddingDropout(nn.Module):
#     "Apply dropout in the embedding layer by zeroing out some elements of the embedding vector."

#     def __init__(self, emb, embed_p:float):
#         super().__init__()
#         self.collection = {}
#         self.emb,self.embed_p = emb,embed_p
#         self.pad_idx = self.emb.padding_idx
#         if self.pad_idx is None: self.pad_idx = -1

#     def forward(self, words:LongTensor, scale:Optional[float]=None) -> Tensor:
#         if self.training and self.embed_p != 0:
#             size = (self.emb.weight.size(0),1)
#             mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
#             masked_embed = self.emb.weight * mask
#         else: masked_embed = self.emb.weight
#         if scale: masked_embed.mul_(scale)
#         return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
#         self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

class RNN_Encoder(nn.Module):

    """A custom RNN encoder network that uses
        - an embedding matrix to encode input,
        - a stack of LSTM or QRNN layers to drive the network, and
        - variational dropouts in the embedding and LSTM/QRNN layers
        The architecture for this network was inspired by the work done in
        "Regularizing and Optimizing LSTM Language Models".
        (https://arxiv.org/pdf/1708.02182.pdf)
    """

    initrange=0.1

    def __init__(self, ntoken, emb_sz, n_hid, n_layers, pad_token, bidir=False,
                 dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5, qrnn=False):
        """ Default constructor for the RNN_Encoder class
            Args:
                bs (int): batch size of input data
                ntoken (int): number of vocabulary (or tokens) in the source dataset
                emb_sz (int): the embedding size to use to encode each token
                n_hid (int): number of hidden activation per LSTM layer
                n_layers (int): number of LSTM layers to use in the architecture
                pad_token (int): the int value used for padding text.
                dropouth (float): dropout to apply to the activations going from one LSTM layer to another
                dropouti (float): dropout to apply to the input layer.
                dropoute (float): dropout to apply to the embedding layer.
                wdrop (float): dropout used for a LSTM's internal (or hidden) recurrent weights.
            Returns:
                None
          """

        super().__init__()
        self.ndir = 2 if bidir else 1
        self.bs, self.qrnn = 1, qrnn
        self.pad_token =pad_token
        self.encoder = nn.Embedding(ntoken, emb_sz, padding_idx=pad_token)
        self.encoder_with_dropout = EmbeddingDropout(self.encoder)
        if self.qrnn:
            #Using QRNN requires cupy: https://github.com/cupy/cupy
            from .torchqrnn.qrnn import QRNNLayer
            self.rnns = [QRNNLayer(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                save_prev_x=True, zoneout=0, window=2 if l == 0 else 1, output_gate=True) for l in range(n_layers)]
            if wdrop:
                for rnn in self.rnns:
                    rnn.linear = WeightDrop(rnn.linear, wdrop, weights=['weight'])
        else:
            self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                1, bidirectional=bidir) for l in range(n_layers)]
            if wdrop: self.rnns = [WeightDrop(rnn, wdrop) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)

        self.emb_sz,self.n_hid,self.n_layers,self.dropoute = emb_sz,n_hid,n_layers,dropoute
        self.dropouti = LockedDropout(dropouti)
        self.dropouths = nn.ModuleList([LockedDropout(dropouth) for l in range(n_layers)])

    def forward(self, input):
        """ Invoked during the forward propagation of the RNN_Encoder module.
        Args:
            input (Tensor): input of shape (sentence length x batch_size)
        Returns:
            raw_outputs (tuple(list (Tensor), list(Tensor)): list of tensors evaluated from each RNN layer without using
            dropouth, list of tensors evaluated from each RNN layer using dropouth,
        """
        sl,bs = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        with set_grad_enabled(self.training):
            mask = tu.compute_mask(input.transpose(1, 0))
            
            emb = self.encoder_with_dropout(input, dropout=self.dropoute if self.training else 0)
#             emb = self.encoder_with_dropout(input)
            emb = self.dropouti(emb)
#             print("emb shape is ", emb.shape)

                    
            lengths = mask.eq(1).long().sum(1)                                    # bs
            lengths_sort, idx_sort = torch.sort(lengths, dim=0, descending=True)  # bs
            _, idx_unsort = torch.sort(idx_sort, dim=0)                           # bs

            emb_sort = emb.index_select(1, idx_sort)    # sl * bs * ninp
            hid_sort = [(h[0].index_select(1, idx_sort), h[1].index_select(1, idx_sort)) for h in self.hidden]
            emb_sort = torch.nn.utils.rnn.pack_padded_sequence(emb_sort, lengths_sort)

            
#             raw_output = emb
            new_hidden,raw_outputs,outputs,raw_outputs_sorted = [],[],[],[]
    
            for l, (rnn,drop) in enumerate(zip(self.rnns, self.dropouths)):
                current_input = emb_sort
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
#                     print("shape of raw output is", raw_output.shape)
                    emb_sort, new_h = rnn(emb_sort, hid_sort[l])

                emb_sort, _ = torch.nn.utils.rnn.pad_packed_sequence(emb_sort)
                new_hidden.append(new_h)
                raw_outputs.append(emb_sort)
                if l != self.n_layers - 1: 
                    emb_sort = drop(emb_sort)
                    outputs.append(emb_sort)
                
#             self.hidden = repackage_var(new_hidden)
            output =  self.dropouths[-1](emb_sort)
            outputs.append(output)
        
            raw_outputs = [raw_output.index_select(1, idx_unsort) for raw_output in raw_outputs]
            outputs = [output.index_select(1, idx_unsort) for output in outputs]
            new_hidden = [(h_sort[0].index_select(1, idx_unsort), h_sort[1].index_select(1, idx_unsort)) for h_sort in new_hidden]
    #         output = self.lockdrop(emb_sort, self.dropout)
    #         outputs.append(output)

        return raw_outputs, new_hidden, outputs

    def one_hidden(self, l):
#         print(self.bs)
        nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz)//self.ndir
        if IS_TORCH_04: return Variable(self.weights.new(self.ndir, self.bs, nh).zero_())
        else: return Variable(self.weights.new(self.ndir, self.bs, nh).zero_(), volatile=not self.training)

    def reset(self):
        if self.qrnn: [r.reset() for r in self.rnns]
        self.weights = next(self.parameters()).data
        if self.qrnn: self.hidden = [self.one_hidden(l) for l in range(self.n_layers)]
        else: self.hidden = [(self.one_hidden(l), self.one_hidden(l)) for l in range(self.n_layers)]
    
    def reset_hidden(self):
        self.hidden = [(self.one_hidden(l), self.one_hidden(l)) for l in range(self.n_layers)]

class PoolingLinearClassifier(nn.Module):
    
    def __init__(self, layers, drops):
        super().__init__()
        print("done")
#         self.layers = nn.ModuleList([
#             LinearBlock(layers[i], layers[i + 1], drops[i]) for i in range(len(layers) - 1)])

    def pool(self, x, bs, is_max):
        f = F.adaptive_max_pool1d if is_max else F.adaptive_avg_pool1d
        return f(x.permute(1,2,0), (1,)).view(bs,-1)

    def forward(self, input):
        raw_outputs, outputs = input
        output = outputs[-1]
        sl,bs,_ = output.size()
        avgpool = self.pool(output, bs, False)
        mxpool = self.pool(output, bs, True)
        x = torch.cat([output[-1], mxpool, avgpool], 1)
        return [output[-1]]
    
class RNN_Encoder_Masks(nn.Module):
    initrange=0.1

    def __init__(self, vocab_sz:int, emb_sz:int, n_hid:int, n_layers:int, pad_token:int, bidir:bool=False,
                 hidden_p:float=0.2, input_p:float=0.6, embed_p:float=0.1, weight_p:float=0.5, qrnn:bool=False):

        super().__init__()
        self.collection = {}
        self.bs,self.qrnn,self.ndir = 1, qrnn,(2 if bidir else 1)
        self.emb_sz,self.n_hid,self.n_layers,self.pad_token = emb_sz,n_hid,n_layers,pad_token
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        if self.qrnn:
            #Using QRNN requires cupy: https://github.com/cupy/cupy
            from .qrnn.qrnn import QRNNLayer
            self.rnns = [QRNNLayer(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                                   save_prev_x=True, zoneout=0, window=2 if l == 0 else 1, output_gate=True,
                                   use_cuda=torch.cuda.is_available()) for l in range(n_layers)]
            if weight_p != 0.:
                for rnn in self.rnns:
                    rnn.linear = WeightDropout(rnn.linear, weight_p, layer_names=['weight'])
        else:
            self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                1, bidirectional=bidir) for l in range(n_layers)]
            if weight_p != 0.: self.rnns = [WeightDropout(rnn, weight_p) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input:LongTensor):
        if is_listy(input):
            input, lengths = input
        else:
            input, lengths = pad_packed_sequence(pack_sequence(input))


        # pack_padded can not handle zero lengths texts
        empty_inputs = np.argwhere(lengths <= 0)
        lengths[empty_inputs] = 1

        sl, bs = input.size()
        if bs != self.bs:
            self.bs = bs
            self.reset()

        raw_output = self.input_dp(self.encoder_dp(input))
        new_hidden, raw_outputs, outputs = [], [], []
        for l, (rnn, hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):

            packed_rnn_inp = pack_padded_sequence(raw_output, lengths)
            rnn_output, new_h = rnn(packed_rnn_inp, self.hidden[l])
            raw_output, _ = pad_packed_sequence(rnn_output)
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
#         self.hidden = _repackage_var(new_hidden)
        return raw_outputs, outputs, lengths

    def _one_hidden(self, l:int) -> Tensor:
        "Return one hidden state."
        nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz)//self.ndir
        return self.weights.new(self.ndir, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.weights = next(self.parameters()).data
        if self.qrnn: self.hidden = [self._one_hidden(l) for l in range(self.n_layers)]
        else: self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]
    
    def reset_hidden(self):
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]
            
class PoolingLinearClassifierMask(nn.Module):
    "Create a linear classifier with pooling."

    def __init__(self, layers:Collection[int], drops:Collection[float]):
        super().__init__()
        mod_layers = []
#         activs = [nn.ReLU(inplace=True)] * (len(layers) - 2) + [None]
#         for n_in,n_out,p,actn in zip(layers[:-1],layers[1:], drops, activs):
#             mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)
#         self.layers = nn.Sequential(*mod_layers)

    def avg_pool(self, x:Tensor, lengths:LongTensor):
        diffs_from_max = lengths[0] - lengths
        avg_lengths = np.minimum(x.shape[0] - diffs_from_max, lengths)
        return torch.div(torch.sum(x, dim=0).permute(1, 0), avg_lengths.float()).permute(1, 0)

    def max_pool(self, x:Tensor, bs:int):
        "Pool the tensor along the seq_len dimension."
        return F.adaptive_max_pool1d(x.permute(1,2,0), (1,)).view(bs,-1)

    def last_output(self, outputs:Tensor, lengths:Collection[float]):
        diffs_from_max = lengths[0] - lengths
        batch_lengths = outputs.shape[0] - diffs_from_max
        return outputs[batch_lengths - 1, np.arange(outputs.shape[1]), :]

    def forward(self, input:Tuple[Tensor,Tensor, LongTensor]) -> Tuple[Tensor,Tensor,Tensor]:
        raw_outputs, outputs, lengths = input
        output = outputs[-1]
        sl,bs,_ = output.size()
#         avgpool = self.avg_pool(output, lengths)
#         mxpool = self.max_pool(output, bs)
        return [self.last_output(outputs[-1].transpose(1,0), lengths)]




In [9]:
# encoder = RNN_Encoder_Masks(vocab_sz=1000, emb_sz=400, weight_p=0.0,
#                                                  n_hid=256, n_layers=2, pad_token=0, qrnn=False).to(device)
# dropout = LockedDropout(0.1)
# classifier = PoolingLinearClassifierMask(layers=[400*3, 50, 300], drops=[0.4, 0.1]).to(device)
# s = 8
# b = 11
# x = torch.randint(0,1000,(b,s))
# x = torch.tensor(x.transpose(1,0),dtype=torch.long,device=device)
# output = classifier(encoder(x))
# output[0].shape

In [10]:
class Model(object):
    """
        Boilerplate class which helps others have some common functionality.
        These are made with some debugging/loading and with corechains in mind

    """

    def prepare_save(self):
        pass

    def load_from(self, location):
        # Pull the data from disk
        model_dump = torch.load(location)

        # Load parameters
        for key in self.prepare_save():
            key[1].load_state_dict(model_dump[key[0]])

    def get_parameter_sum(self):

        sum = 0
        for model in self.prepare_save():

            model_sum = 0
            for x in list(model[1].parameters()):

                model_sum += np.sum(x.data.cpu().numpy().flatten())

            sum += model_sum

        return sum

    def freeze_layer(self,layer):
        for params in layer.parameters():
            params.requires_grad = False

    def unfreeze_layer(self,layer):
        for params in layer.parameters():
            params.requires_grad = True

In [11]:
class BiLstmDot_ulmfit(Model):

    def __init__(self, _parameter_dict, _word_to_id, _device, _pointwise=False, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device
        self.pointwise = _pointwise
        self.word_to_id = _word_to_id

        if self.debug:
            print("Init Models")

        # Load the pre-trained model
        pretrained_weights = torch.load('./ulmfit/wt103/fwd_wt103_enc.h5', map_location= lambda storage, loc: storage)
        new_vectors = self.parameter_dict['vectors']
        pretrained_weights['encoder.weight'] = T(new_vectors)
        pretrained_weights.pop('encoder_with_dropout.embed.weight')
#         pretrained_weights['encoder_with_dropout.embed.weight'] = T(np.copy(new_vectors))

        # self.encoder = fastai.old.lm_rnn.RNN_Encoder(ntoken=new_vectors.shape[0], emb_sz=400, n_hid=1150, n_layers=1, pad_token=0,qrnn=False).to(self.device)
        # self.encoder.load_state_dict(pretrained_weights)
        # fastai.RNNTrainer
        # self.encoder = com.NotSuchABetterEncoder(
        #     number_of_layer=self.parameter_dict['number_of_layer'],
        #     bidirectional=self.parameter_dict['bidirectional'],
        #     embedding_dim=self.parameter_dict['embedding_dim'],
        #     max_length = self.parameter_dict['max_length'],
        #     hidden_dim=self.parameter_dict['hidden_size'],
        #     vocab_size=self.parameter_dict['vocab_size'],
        #     dropout=self.parameter_dict['dropout'],
        #     vectors=self.parameter_dict['vectors'],
        #     enable_layer_norm=False,
        #     mode = 'LSTM',
        #     debug = self.debug).to(self.device)

#         self.encoder = RNN_Encoder(ntoken=self.parameter_dict['vectors'].shape[0], emb_sz=400,
#                                                  n_hid=512, n_layers=1, pad_token=0, qrnn=False)
  
                                   #Merity
        self.encoder = merity.RNNModel(rnn_type='LSTM', 
                ntoken=self.parameter_dict['vectors'].shape[0],
                ninp=400,
                nhid=1150,
                nlayers=3,
                dropout=0.1,
                dropouth=0.5,
                dropouti=0.5,
                dropoute=0.5,
                wdrop=0,
                tie_weights=True)
        self.encoder.encoder.padding_idx = 0
        
        
        key_mapping = {
            'encoder.weight' : 'encoder.weight',
            'rnns.0.module.weight_ih_l0' : 'rnns.0.weight_ih_l0', 
            'rnns.0.module.bias_ih_l0' : 'rnns.0.bias_ih_l0', 
            'rnns.0.module.bias_hh_l0' : 'rnns.0.bias_hh_l0', 
            'rnns.0.module.weight_hh_l0_raw' : 'rnns.0.weight_hh_l0', 
            'rnns.1.module.weight_ih_l0' : 'rnns.1.weight_ih_l0', 
            'rnns.1.module.bias_ih_l0' : 'rnns.1.bias_ih_l0', 
            'rnns.1.module.bias_hh_l0': 'rnns.1.bias_hh_l0', 
            'rnns.1.module.weight_hh_l0_raw' : 'rnns.1.weight_hh_l0', 
            'rnns.2.module.weight_ih_l0' : 'rnns.2.weight_ih_l0', 
            'rnns.2.module.bias_ih_l0' : 'rnns.2.bias_ih_l0', 
            'rnns.2.module.bias_hh_l0': 'rnns.2.bias_hh_l0', 
            'rnns.2.module.weight_hh_l0_raw' : 'rnns.2.weight_hh_l0',
        }
        
        
        for k,v in key_mapping.items():
            pretrained_weights[v] = pretrained_weights.pop(k)
        
        
#         self.encoder.encoder = nn.Embedding.from_pretrained(torch.FloatTensor(self.parameter_dict['vectors']))
        self.encoder = self.encoder.to(self.device)
         
        
#         self.encoder = RNN_Encoder_Masks(vocab_sz=self.parameter_dict['vectors'].shape[0], emb_sz=400, weight_p=0,
#                                                  n_hid=256, n_layers=1, pad_token=0, qrnn=False).to(device)
        self.encoder.load_state_dict(pretrained_weights)
#         self.encoder.encoder.weight = torch.tensor(self.parameter_dict['vectors'],device=self.device)
#         self.encoder.encoder = nn.Embedding.from_pretrained(torch.FloatTensor(self.parameter_dict['vectors']))
        
#         self.classifier = PoolingLinearClassifier(layers=[400*3, 50, 300], drops=[0.4, 0.1]).to(device)
#         self.classifier = self.classifier.to(device)
        
#         self.encoder = self.encoder.to(self.device)
        
        self.encoder.reset()
    
        # self.linear = torch.nn.Linear(1150,256).to(self.device)

    def train(self, data, optimizer, loss_fn, device):
        if self.pointwise:
            return self._train_pointwise_(data, optimizer, loss_fn, device)
        else:
            return self._train_pairwise_(data, optimizer, loss_fn, device)

    def _train_pointwise_(self, data, optimizer, loss_fn, device):
        self.encoder.train()
        self.classifier.train()        
        optimizer.zero_grad()
        self.encoder.reset_hidden()
        
        ques_batch, path_batch, y_label = data['ques_batch'], data['path_batch'], data['y_label']
        self.encoder.reset_hidden()
        op_q = self.encoder(ques_batch.transpose(1,0))
        self.encoder.reset_hidden()
        op_p = self.encoder(path_batch.transpose(1,0))
        
        ques_batch_encoded =  op_q
        path_batch_encoded =  op_p
        
         #Calculating dot score
        scores = torch.sum(ques_batch_encoded * path_batch_encoded, -1)

        try:
            loss = loss_fn(scores, y_label)
        except RuntimeError:
            traceback.print_exc()
            print(pos_scores.shape, neg_scores.shape, y_label.shape,  ques_batch.shape, pos_batch.shape, neg_batch.shape)

        loss.backward()
        optimizer.step()
        return loss
        
        
    def _train_pairwise_(self, data, optimizer, loss_fn, device):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, pos paths, neg paths and dummy y labels}
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object

            returns loss
        '''
        self.encoder.train()
        # Unpacking the data and model from args
        ques_batch, pos_batch, neg_batch, y_label = data['ques_batch'], data['pos_batch'], data['neg_batch'], data['y_label']
        
        optimizer.zero_grad()
        #Encoding all the data

#         print(f"Before: \t {ques_batch.shape[1]}, {pos_batch.shape[1]}, {neg_batch.shape[1]}\n"
#         f"After: \t\t {tu.trim(ques_batch).shape[1]}, {tu.trim(pos_batch).shape[1]}, {tu.trim(neg_batch).shape[1]}")
        
        ques_batch = tu.trim(ques_batch)
        pos_batch = tu.trim(pos_batch)
        neg_batch = tu.trim(neg_batch)
        
#         print(pos_batch)
#         print(neg_batch)

        h = self.encoder.init_hidden(ques_batch.shape[0])
        op_q = self.encoder(ques_batch.transpose(1,0), h)[1][-1][0].squeeze()
        op_p = self.encoder(pos_batch.transpose(1,0), h)[1][-1][0].squeeze()
        op_n = self.encoder(neg_batch.transpose(1,0), h)[1][-1][0].squeeze()
                                   
        
#         self.encoder.reset_hidden()
#         op_q = self.encoder(ques_batch.transpose(1,0))[-1][-1][-1]
#         self.encoder.reset_hidden()
#         op_p = self.encoder(pos_batch.transpose(1,0))[-1][-1][-1]
#         self.encoder.reset_hidden()
#         op_n = self.encoder(neg_batch.transpose(1,0))[-1][-1][-1]
#         self.encoder.reset_hidden()
#         op_q = self.encoder(ques_batch.transpose(1,0))[1][-1][0].squeeze()
#         self.encoder.reset_hidden()
#         op_p = self.encoder(pos_batch.transpose(1,0))[1][-1][0].squeeze()
#         self.encoder.reset_hidden()
#         op_n = self.encoder(neg_batch.transpose(1,0))[1][-1][0].squeeze()

#         print(op_q.shape, op_p.shape, op_n.shape)
#         raise IOError
            
        ques_batch_encoded =  op_q
        pos_batch_encoded =  op_p
        neg_batch_encoded =  op_n
        
#         ques_batch_encoded =  op_q[-1][0][1].squeeze()
#         pos_batch_encoded =  op_p[-1][0][1].squeeze()
#         neg_batch_encoded =  op_n[-1][0][1].squeeze()
        
#         print(ques_batch_encoded.shape, pos_batch_encoded.shape, neg_batch_encoded.shape)
#         raise IOError
        
        #Calculating dot score
        pos_scores = torch.sum(ques_batch_encoded * pos_batch_encoded, -1)
        neg_scores = torch.sum(ques_batch_encoded * neg_batch_encoded, -1)
#         print(pos_scores, neg_scores)
        
#         raise IOError
        
#         if True:
#             print("ques_batch shape is ", ques_batch.shape)
#             print("pos_batch shape is ", pos_batch.shape)
#             print("neg_batch shape is ", neg_batch.shape)
#             print("transposed ques bathc is ", ques_batch.transpose(1,0).shape)
#             for o in op_p[1]:
#                 print("o shape is ", o.shape)
#             print("encoded pos batch shape is ", op_p[1][-1][-1].shape)
            
#             print("pos_score is", pos_scores.shape)
        '''
            If `y == 1` then it assumed the first input should be ranked higher
            (have a larger value) than the second input, and vice-versa for `y == -1`
        '''
#         raise ValueError
        try:
            loss = loss_fn(pos_scores, neg_scores, y_label)
        except RuntimeError:
            traceback.print_exc()
            print(pos_scores.shape, neg_scores.shape, y_label.shape,  ques_batch.shape, pos_batch.shape, neg_batch.shape)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        optimizer.step()
        return loss

    def predict(self, ques, paths, device):
        """
            Same code works for both pairwise or pointwise
        """
        with torch.no_grad():

#             print(ques.shape, paths.shape, tu.trim(ques).shape,tu.trim(paths).shape)
            
#             if tu.trim(ques).shape[0] == 0:
#                 print(ques)
#                 print(paths)
            
            self.encoder.eval()
#             h = self.encoder.reset()
            # Encoding all the data
            ques = tu.trim(ques)
            paths = tu.trim(paths)
    
            
            h  = self.encoder.init_hidden(ques.shape[0])
            op_q = self.encoder(ques.transpose(1,0), h)
            op_p = self.encoder(paths.transpose(1,0), h)
#             self.encoder.reset_hidden()
#             op_q = self.encoder(ques.transpose(1,0))
#             self.encoder.reset_hidden()
#             op_p = self.encoder(paths.transpose(1,0))

#             question = op_q[1][0][-1]
#             paths = op_p[1][0][-1]
            question = op_q[1][-1][0].squeeze()
            paths = op_p[1][-1][0].squeeze()
        
#             question = op_q[-1][-1][-1]
#             paths = op_p[-1][-1][-1]
        
#             question = op_q[-1][0][1].squeeze()
#             paths = op_p[-1][0][1].squeeze()

            if self.pointwise:
                # question = F.normalize(F.relu(question),p=1,dim=1)
                # paths = F.normalize(F.relu(paths),p=1,dim=1)
                # norm_ques_batch = torch.abs(torch.norm(question, dim=1, p=1))
                # norm_pos_batch = torch.abs(torch.norm(paths, dim=1, p=1))
                score = torch.sum(question * paths, -1)
                # score = score.div(norm_ques_batch * norm_pos_batch).div_(2.0).add_(0.5)
            else:
                score = torch.sum(question * paths, -1)

            self.encoder.train()
            return score
    
    def prepare_save(self):
        """

            This function is called when someone wants to save the underlying models.
            Returns a tuple of key:model pairs which is to be interpreted within save model.

        :return: [(key, model)]
        """
        return [('encoder', self.encoder)]

    def load_from(self, location):
        # Pull the data from disk
        if self.debug: print("loading Bilstmdot model from", location)
        self.encoder.load_state_dict(torch.load(location)['encoder'])
        if self.debug: print("model loaded with weights ,", self.get_parameter_sum())

In [12]:
class BiLstmDot(Model):

    def __init__(self, _parameter_dict, _word_to_id, _device, _pointwise=False, _debug=False):

        self.debug = _debug
        self.parameter_dict = _parameter_dict
        self.device = _device
        self.pointwise = _pointwise
        self.word_to_id = _word_to_id

        if self.debug:
            print("Init Models")


        self.encoder = com.Encoder(self.parameter_dict['max_length'], self.parameter_dict['hidden_size'],
                                   self.parameter_dict['number_of_layer'], self.parameter_dict['embedding_dim'],
                                   self.parameter_dict['vocab_size'],
                                   bidirectional=self.parameter_dict['bidirectional'],
                                   vectors=self.parameter_dict['vectors']).to(self.device)            

#         self.encoder = com.NotSuchABetterEncoder(
#             number_of_layer=self.parameter_dict['number_of_layer'],
#             bidirectional=self.parameter_dict['bidirectional'],
#             embedding_dim=self.parameter_dict['embedding_dim'],
#             max_length = self.parameter_dict['max_length'],
#             hidden_dim=self.parameter_dict['hidden_size'],
#             vocab_size=self.parameter_dict['vocab_size'],
#             dropout=self.parameter_dict['dropout'],
#             vectors=self.parameter_dict['vectors'],
#             enable_layer_norm=False,
#             mode = 'LSTM',
#             debug = self.debug).to(self.device)

    def train(self, data, optimizer, loss_fn, device):
    #
        if self.pointwise:
            return self._train_pointwise_(data, optimizer, loss_fn, device)
        else:
            return self._train_pairwise_(data, optimizer, loss_fn, device)

    def _train_pairwise_(self, data, optimizer, loss_fn, device):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, pos paths, neg paths and dummy y labels}
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object

            returns loss
        '''

        # Unpacking the data and model from args
        ques_batch, pos_batch, neg_batch, y_label = data['ques_batch'], data['pos_batch'], data['neg_batch'], data['y_label']

        optimizer.zero_grad()
        #Encoding all the data

        hidden = self.encoder.init_hidden(ques_batch.shape[0],self.device)
        ques_batch_encoded, _ = self.encoder(tu.trim(ques_batch), hidden)
        pos_batch_encoded, _ = self.encoder(tu.trim(pos_batch), hidden)
        neg_batch_encoded, _  = self.encoder(tu.trim(neg_batch), hidden)



        #Calculating dot score
        pos_scores = torch.sum(ques_batch_encoded[-1] * pos_batch_encoded[-1], -1)
        neg_scores = torch.sum(ques_batch_encoded[-1] * neg_batch_encoded[-1], -1)
        '''
            If `y == 1` then it assumed the first input should be ranked higher
            (have a larger value) than the second input, and vice-versa for `y == -1`
        '''
        try:
            loss = loss_fn(pos_scores, neg_scores, y_label)
        except RuntimeError:
            print(pos_scores.shape, neg_scores.shape, y_label.shape,  ques_batch.shape, pos_batch.shape, neg_batch.shape)
        loss.backward()
#         torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        optimizer.step()
        return loss

    def _train_pointwise_(self, data, optimizer, loss_fn, device):
        '''
            Given data, passes it through model, inited in constructor, returns loss and updates the weight
            :params data: {batch of question, paths and y labels}
            :params models list of [models]
            :params optimizer: torch.optim object
            :params loss fn: torch.nn loss object
            :params device: torch.device object
            returrns loss
        '''
        # Unpacking the data and model from args
        ques_batch, path_batch, y_label = data['ques_batch'], data['path_batch'], data['y_label']

        optimizer.zero_grad()

        # Encoding all the data
        hidden = self.encoder.init_hidden(ques_batch.shape[0], self.device)
        _, ques_batch, _, _ = self.encoder(tu.trim(ques_batch), hidden)
        _, pos_batch, _, _ = self.encoder(tu.trim(path_batch), hidden)

        #
        # norm_ques_batch = torch.abs(torch.norm(ques_batch,dim=1,p=1))
        # norm_pos_batch = torch.abs(torch.norm(pos_batch,dim=1,p=1))

        # ques_batch = F.normalize(F.relu(ques_batch),p=1,dim=1)
        # pos_batch = F.normalize(F.relu(pos_batch),p=1,dim=1)
        # ques_batch =(F.normalize(ques_batch,p=1,dim=1)/2) + .5
        # pos_batch =(F.normalize(pos_batch,p=1,dim=1)/2) + .5




        # Calculating dot score
        score = torch.sum(ques_batch * pos_batch, -1)
        # score = score.div(norm_ques_batch*norm_pos_batch).div_(2.0).add_(0.5)
            # print("shape of score is,", score.shape)
            # print("score is , ", score)
            #
            #
            # print("shape of y label is ", y_label.shape)
            # print("value of y label is ", y_label)

        # raise ValueError

        '''
            Binary Cross Entropy loss function. @TODO: Check if we can give it 1/0 labels.
        '''
        loss = loss_fn(score, y_label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), .5)
        optimizer.step()

        return loss

    def predict(self, ques, paths, device):
        """
            Same code works for both pairwise or pointwise
        """
        with torch.no_grad():

            self.encoder.eval()
            hidden = self.encoder.init_hidden(ques.shape[0], self.device)

            question, _ = self.encoder(tu.trim(ques.long()), hidden)
            paths, _ = self.encoder(tu.trim(paths.long()), hidden)

            if self.pointwise:
                # question = F.normalize(F.relu(question),p=1,dim=1)
                # paths = F.normalize(F.relu(paths),p=1,dim=1)
                # norm_ques_batch = torch.abs(torch.norm(question, dim=1, p=1))
                # norm_pos_batch = torch.abs(torch.norm(paths, dim=1, p=1))
                score = torch.sum(question[-1] * paths[-1], -1)
                # score = score.div(norm_ques_batch * norm_pos_batch).div_(2.0).add_(0.5)
            else:
                score = torch.sum(question[-1] * paths[-1], -1)

            self.encoder.train()
            return score

    def prepare_save(self):
        """

            This function is called when someone wants to save the underlying models.
            Returns a tuple of key:model pairs which is to be interpreted within save model.

        :return: [(key, model)]
        """
        return [('encoder', self.encoder)]

    def load_from(self, location):
        # Pull the data from disk
        if self.debug: print("loading Bilstmdot model from", location)
        self.encoder.load_state_dict(torch.load(location)['encoder'])
        if self.debug: print("model loaded with weights ,", self.get_parameter_sum())

In [13]:
parameter_dict['vectors'] = data['vectors']
parameter_dict['schema'] = schema

parameter_dict['bidirectional'] = False

In [14]:
parameter_dict['batch_size'] = 100

In [15]:
modeler = BiLstmDot_ulmfit(_parameter_dict = parameter_dict,
                    _word_to_id=_word_to_id,
                    _device=device,
                    _pointwise=pointwise,
                    _debug=False)

optimizer = optim.Adam(list(filter(lambda p: p.requires_grad, modeler.encoder.parameters())))
#                       list(filter(lambda p: p.requires_grad, modeler.classifier.parameters())))

[LSTM(400, 1150), LSTM(1150, 1150), LSTM(1150, 400)]


In [16]:
def training_loop(training_model, parameter_dict,modeler,train_loader,
                  optimizer,loss_func, data, dataset, device, test_every, validate_every , pointwise = False, problem='core_chain',curtail_padding_rel=True):

    model_save_location = aux.save_location(problem, training_model, dataset)
    aux_save_information = {
        'epoch' : 0,
        'test_accuracy':0.0,
        'validation_accura""cy':0.0,
        'parameter_dict':parameter_dict
    }
    train_loss = []
    valid_accuracy = []
    test_accuracy = []
    best_validation_accuracy = 0
    best_test_accuracy = 0

    if parameter_dict['schema'] == 'reldet':
        parameter_dict['rel1_pad'] =  parameter_dict['relrd_pad']
    elif parameter_dict['schema'] == 'slotptr':
        parameter_dict['rel1_pad'] = parameter_dict['relsp_pad']

        ###############
    # Training Loop
    ###############


    #Makes test data of appropriate shape
    print("the dataset is ", dataset)
    if curtail_padding_rel and dataset == 'lcquad':
        data = cc.curatail_padding(data, parameter_dict)
        data['valid_neg_paths'] = np.zeros_like(data['valid_neg_paths'])
        data['valid_pos_paths'] = np.zeros_like(data['valid_pos_paths'])
        data['valid_questions'] = np.zeros_like(data['valid_questions'])

    try:

        for epoch in range(parameter_dict['epochs']):

            # Epoch start print
            print("Epoch: ", epoch, "/", parameter_dict['epochs'])

            # Bookkeeping variables
            epoch_loss = []
            epoch_time = time.time()

            # Loop for one batch
            # tqdm_loop = tqdm(enumerate(train_loader))
            for i_batch, sample_batched in enumerate(train_loader):

                # Bookkeeping and data preparation
                batch_time = time.time()

                if not pointwise:
                    ques_batch = torch.tensor(np.reshape(sample_batched[0][0], (-1, parameter_dict['max_length'])),
                                              dtype=torch.long, device=device)
                    pos_batch = torch.tensor(np.reshape(sample_batched[0][1], (-1, parameter_dict['max_length'])),
                                             dtype=torch.long, device=device)
                    neg_batch = torch.tensor(np.reshape(sample_batched[0][2], (-1, parameter_dict['max_length'])),
                                             dtype=torch.long, device=device)

                    data['dummy_y'] = torch.ones(ques_batch.shape[0], device=device)
                    if parameter_dict['schema'] != 'default':
                        pos_rel1_batch = torch.tensor(np.reshape(sample_batched[0][3], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        pos_rel2_batch = torch.tensor(np.reshape(sample_batched[0][4], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        neg_rel1_batch = torch.tensor(np.reshape(sample_batched[0][5], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        neg_rel2_batch = torch.tensor(np.reshape(sample_batched[0][6], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)


                        data_batch = {
                            'ques_batch': ques_batch,
                            'pos_batch': pos_batch[:,:parameter_dict['rel_pad']],
                            'neg_batch': neg_batch[:,:parameter_dict['rel_pad']],
                            'y_label': data['dummy_y'],
                            'pos_rel1_batch': pos_rel1_batch[:,:parameter_dict['rel1_pad']],
                            'pos_rel2_batch':pos_rel2_batch[:,:parameter_dict['rel1_pad']],
                            'neg_rel1_batch':neg_rel1_batch[:,:parameter_dict['rel1_pad']],
                            'neg_rel2_batch' : neg_rel2_batch[:,:parameter_dict['rel1_pad']]
                        }

                    else:

                        data_batch = {
                            'ques_batch': ques_batch,
                            'pos_batch': pos_batch[:,:parameter_dict['rel_pad']],
                            'neg_batch': neg_batch[:,:parameter_dict['rel_pad']],
                            'y_label': data['dummy_y']}

                else:
                    ques_batch = torch.tensor(np.reshape(sample_batched[0][0], (-1, parameter_dict['max_length'])),
                                              dtype=torch.long, device=device)
                    path_batch = torch.tensor(np.reshape(sample_batched[0][1], (-1, parameter_dict['max_length'])),
                                             dtype=torch.long, device=device)
                    y = torch.tensor(sample_batched[1],dtype = torch.float,device=device).view(-1)


                    if parameter_dict['schema'] != 'default':
                        path_rel1_batch = torch.tensor(np.reshape(sample_batched[0][2], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)
                        path_rel2_batch = torch.tensor(np.reshape(sample_batched[0][3], (-1, parameter_dict['max_length'])),
                                                      dtype=torch.long, device=device)

                        data_batch = {
                            'ques_batch': ques_batch,
                            'path_batch': path_batch[:,:parameter_dict['rel_pad']],
                            'y_label': y,
                            'path_rel1_batch': path_rel1_batch[:,:parameter_dict['rel1_pad']],
                            'path_rel2_batch': path_rel2_batch[:,:parameter_dict['rel1_pad']]
                        }
                    else:
                        data_batch = {
                            'ques_batch': ques_batch,
                            'path_batch': path_batch[:,:parameter_dict['rel_pad']],
                            'y_label': y
                        }


                return data_batch
                loss = modeler.train(data=data_batch,
                                  optimizer=optimizer,
                                  loss_fn=loss_func,
                                  device=device)

                # Bookkeep the training loss
                epoch_loss.append(loss.item())

                # tqdm_loop.desc("#"+str(i_batch)+"\tLoss:" + str(loss.item())[:min(5, len(str(loss.item())))])

                print("Batch:\t%d" % i_batch, "/%d\t: " % (parameter_dict['batch_size']),
                      "%s" % (time.time() - batch_time),
                      "\t%s" % (time.time() - epoch_time),
                      "\t%s" % (str(loss.item())),
                      end=None if i_batch + 1 == int(int(i_batch) / parameter_dict['batch_size']) else "\n")

            # EPOCH LEVEL

            # Track training loss
            train_loss.append(sum(epoch_loss))

            # test_every = False
            if test_every:
                # Run on test set
                if epoch%test_every == 0:
                    if parameter_dict['schema'] != 'default':
                        if parameter_dict['schema']  == 'slotptr':
                            test_accuracy.append(aux.validation_accuracy(data['test_questions'], data['test_pos_paths'],
                                                             data['test_neg_paths'],modeler, device,data['test_pos_paths_rel1_sp'],data['test_pos_paths_rel2_sp'],
                                                                 data['test_neg_paths_rel1_sp'],data['test_neg_paths_rel2_sp']))
                        else:
                            test_accuracy.append(aux.validation_accuracy(data['test_questions'], data['test_pos_paths'],
                                                                         data['test_neg_paths'], modeler, device,
                                                                         data['test_pos_paths_rel1_rd'],
                                                                         data['test_pos_paths_rel2_rd'],
                                                                         data['test_neg_paths_rel1_rd'],
                                                                         data['test_neg_paths_rel2_rd']))
                    else:
                        test_accuracy.append(aux.validation_accuracy(data['test_questions'], data['test_pos_paths'],
                                                                     data['test_neg_paths'], modeler, device))
                    if test_accuracy[-1] >= best_test_accuracy:
                        best_test_accuracy = test_accuracy[-1]
                        aux_save_information['test_accuracy'] = best_test_accuracy
            else:
                test_accuracy.append(0)
                best_test_accuracy = 0

            # Run on validation set
            if validate_every:
                if epoch%validate_every == 0:
                    if parameter_dict['schema'] != 'default':
                        if parameter_dict['schema'] == 'slotptr':
                            valid_accuracy.append(aux.validation_accuracy(data['valid_questions'], data['valid_pos_paths'],
                                                              data['valid_neg_paths'],  modeler, device, data['valid_pos_paths_rel1_sp'],data['valid_pos_paths_rel2_sp'],
                                                                 data['valid_neg_paths_rel1_sp'],data['valid_neg_paths_rel2_sp']))
                        else:
                            valid_accuracy.append(aux.validation_accuracy(data['valid_questions'][:-1], data['valid_pos_paths'][:-1],
                                                                          data['valid_neg_paths'][:-1], modeler, device,
                                                                          data['valid_pos_paths_rel1_rd'][:-1],
                                                                          data['valid_pos_paths_rel2_rd'][:-1],
                                                                          data['valid_neg_paths_rel1_rd'][:-1],
                                                                          data['valid_neg_paths_rel2_rd'][:-1]))
                    else:
                        valid_accuracy.append(aux.validation_accuracy(data['valid_questions'], data['valid_pos_paths'],
                                                                      data['valid_neg_paths'], modeler, device))
                    if valid_accuracy[-1] > best_validation_accuracy:
                        print("MODEL WEIGHTS RIGHT NOW: ", modeler.get_parameter_sum())
                        best_validation_accuracy = valid_accuracy[-1]
                        aux_save_information['epoch'] = epoch
                        aux_save_information['validation_accuracy'] = best_validation_accuracy
                        aux.save_model(model_save_location, modeler, model_name='model.torch'
                                   , epochs=epoch, optimizer=optimizer, accuracy=best_validation_accuracy, aux_save_information=aux_save_information)

            # Resample new negative paths per epoch and shuffle all data
            train_loader.dataset.shuffle()

            # Epoch level prints
            print("Time: %s\t" % (time.time() - epoch_time),
                  "Loss: %s\t" % (sum(epoch_loss)),
                  "Valdacc: %s\t" % (valid_accuracy[-1]),
                    "Testacc: %s\n" % (test_accuracy[-1]),
                  "BestValidAcc: %s\n" % (best_validation_accuracy),
                  "BestTestAcc: %s\n" % (best_test_accuracy))

        return train_loss, modeler, valid_accuracy, test_accuracy

    except KeyboardInterrupt:
        print('-' * 89)
        return train_loss, modeler, valid_accuracy, test_accuracy


In [ ]:
if not pointwise:
    loss_func = nn.MarginRankingLoss(margin=1, size_average=False)
else:
    loss_func = nn.BCEWithLogitsLoss()
    training_model += '_pointwise'
# db = training_loop(training_model = training_model,
#                    parameter_dict = parameter_dict,
#                    modeler = modeler,
#                    train_loader = train_loader,
#                    optimizer=optimizer,
#                    loss_func=loss_func,
#                    data=data,
#                    dataset=parameter_dict['dataset'],
#                    device=device,
#                    test_every=1,
#                    validate_every=1,
#                     pointwise=pointwise,
#                    problem='core_chain')
train_loss, modeler, valid_accuracy, test_accuracy = cc.training_loop(training_model = training_model,
                                                                           parameter_dict = parameter_dict,
                                                                           modeler = modeler,
                                                                           train_loader = train_loader,
                                                                           optimizer=optimizer,
                                                                           loss_func=loss_func,
                                                                           data=data,
                                                                           dataset=parameter_dict['dataset'],
                                                                           device=device,
                                                                           test_every=1,
                                                                           validate_every=1,
                                                                            pointwise=pointwise,
                                                                           problem='core_chain')

/data/priyansh/conda/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


the dataset is  lcquad
Epoch:  0 / 200
Batch:	0 /100	:  0.30010533332824707 	0.3031270503997803 	486.80352783203125
Batch:	1 /100	:  0.2783393859863281 	0.5825595855712891 	442.3117980957031
Batch:	2 /100	:  0.2769312858581543 	0.8614952564239502 	477.81134033203125
Batch:	3 /100	:  0.28230834007263184 	1.144834041595459 	444.215576171875
Batch:	4 /100	:  0.28002238273620605 	1.426368236541748 	389.7696228027344
Batch:	5 /100	:  0.2731955051422119 	1.701167106628418 	405.7113342285156
Batch:	6 /100	:  0.276566743850708 	1.978806495666504 	372.45196533203125
Batch:	7 /100	:  0.26964521408081055 	2.249575138092041 	317.3166198730469
Batch:	8 /100	:  0.28888583183288574 	2.540247917175293 	320.3661193847656
Batch:	9 /100	:  0.2774317264556885 	2.818896532058716 	386.9849853515625
Batch:	10 /100	:  0.2817087173461914 	3.101989507675171 	279.6360778808594
Batch:	11 /100	:  0.28110694885253906 	3.384559154510498 	366.0398254394531
Batch:	12 /100	:  0.2835969924926758 	3.6697349548339844 	558

Batch:	108 /100	:  0.2802159786224365 	31.038423776626587 	238.60226440429688
Batch:	109 /100	:  0.28276562690734863 	31.322746515274048 	205.26211547851562
Batch:	110 /100	:  0.2862098217010498 	31.610270261764526 	346.9374084472656
Batch:	111 /100	:  0.28723835945129395 	31.898514986038208 	286.8647766113281
Batch:	112 /100	:  0.28548479080200195 	32.1851441860199 	209.57655334472656
Batch:	113 /100	:  0.28124547004699707 	32.46798491477966 	240.33363342285156
Batch:	114 /100	:  0.27350568771362305 	32.74300479888916 	220.9574737548828
Batch:	115 /100	:  0.28095436096191406 	33.02495861053467 	153.85643005371094
Batch:	116 /100	:  0.2843177318572998 	33.310877084732056 	273.86102294921875
Batch:	117 /100	:  0.283980131149292 	33.595834255218506 	159.44247436523438
Batch:	118 /100	:  0.28135108947753906 	33.878387212753296 	226.38348388671875
Batch:	119 /100	:  0.2854924201965332 	34.16508722305298 	215.88113403320312
Batch:	120 /100	:  0.28433656692504883 	34.45069479942322 	339.9752

Batch:	37 /100	:  0.2810359001159668 	10.865610599517822 	129.95530700683594
Batch:	38 /100	:  0.2778646945953369 	11.145063877105713 	178.83370971679688
Batch:	39 /100	:  0.2879598140716553 	11.434070348739624 	149.1237030029297
Batch:	40 /100	:  0.31560683250427246 	11.751316547393799 	220.88011169433594
Batch:	41 /100	:  0.28202176094055176 	12.03441333770752 	191.55474853515625
Batch:	42 /100	:  0.28351902961730957 	12.31952977180481 	126.9154281616211
Batch:	43 /100	:  0.2863035202026367 	12.606808185577393 	142.00152587890625
Batch:	44 /100	:  0.29629993438720703 	12.904948234558105 	117.78494262695312
Batch:	45 /100	:  0.28913331031799316 	13.195143699645996 	132.32952880859375
Batch:	46 /100	:  0.2876136302947998 	13.484155416488647 	108.631103515625
Batch:	47 /100	:  0.2788584232330322 	13.76409387588501 	207.93338012695312
Batch:	48 /100	:  0.2863280773162842 	14.051645994186401 	230.4327850341797
Batch:	49 /100	:  0.2895810604095459 	14.34233546257019 	115.39971160888672
Bat

Batch:	144 /100	:  0.28217196464538574 	41.48791289329529 	89.84453582763672
Batch:	145 /100	:  0.28656721115112305 	41.77557349205017 	145.39210510253906
Batch:	146 /100	:  0.290424108505249 	42.0671706199646 	338.5005798339844
Batch:	147 /100	:  0.2816345691680908 	42.35083556175232 	173.00035095214844
Batch:	148 /100	:  0.28731870651245117 	42.639689207077026 	76.1186752319336
Batch:	149 /100	:  0.28518152236938477 	42.92657494544983 	408.4632873535156
Batch:	150 /100	:  0.2824409008026123 	43.210031270980835 	73.583251953125
Batch:	151 /100	:  0.2798268795013428 	43.49136543273926 	75.49170684814453
Batch:	152 /100	:  0.2822723388671875 	43.775062561035156 	126.57853698730469
Batch:	153 /100	:  0.2846214771270752 	44.06110167503357 	89.33523559570312
Batch:	154 /100	:  0.27750492095947266 	44.33958339691162 	161.671142578125
Batch:	155 /100	:  0.2801227569580078 	44.62152934074402 	333.8701171875
Batch:	156 /100	:  0.28505444526672363 	44.90749931335449 	116.73599243164062
Batch:	1

...

Batch:	149 /100	:  0.28398966789245605 	43.104522466659546 	103.7620849609375
Batch:	150 /100	:  0.284726619720459 	43.3909125328064 	87.00450897216797
Batch:	151 /100	:  0.2762794494628906 	43.668394804000854 	129.5108642578125
Batch:	152 /100	:  0.28727245330810547 	43.95707583427429 	54.47871780395508
Batch:	153 /100	:  0.28612732887268066 	44.24497175216675 	66.72395324707031
Batch:	154 /100	:  0.2836785316467285 	44.53045654296875 	73.89005279541016
Batch:	155 /100	:  0.28339624404907227 	44.81532549858093 	89.38955688476562
Batch:	156 /100	:  0.28037309646606445 	45.09705114364624 	155.76463317871094
Batch:	157 /100	:  0.2805948257446289 	45.378865242004395 	120.76791381835938
Batch:	158 /100	:  0.28336548805236816 	45.663652181625366 	77.0235595703125
Batch:	159 /100	:  0.289264440536499 	45.95396614074707 	132.5420684814453
Batch:	160 /100	:  0.2871890068054199 	46.242701053619385 	117.29647064208984
Batch:	161 /100	:  0.28595590591430664 	46.52989196777344 	81.84172058105469
B

Batch:	80 /100	:  0.28598761558532715 	23.333865880966187 	125.3939437866211
Batch:	81 /100	:  0.2804267406463623 	23.61560082435608 	62.14924621582031
Batch:	82 /100	:  0.28045058250427246 	23.898257732391357 	103.91008758544922
Batch:	83 /100	:  0.2856738567352295 	24.18548011779785 	73.37602996826172
Batch:	84 /100	:  0.2889134883880615 	24.475658893585205 	54.07710266113281
Batch:	85 /100	:  0.2868778705596924 	24.764184713363647 	105.18498992919922
Batch:	86 /100	:  0.2804298400878906 	25.04598307609558 	69.46707916259766
Batch:	87 /100	:  0.285797119140625 	25.333202838897705 	77.98259735107422
Batch:	88 /100	:  0.286759614944458 	25.621427536010742 	72.1031494140625
Batch:	89 /100	:  0.2861766815185547 	25.908990383148193 	94.52405548095703
Batch:	90 /100	:  0.2952420711517334 	26.206372022628784 	75.16038513183594
Batch:	91 /100	:  0.2900075912475586 	26.497451305389404 	101.90608215332031
Batch:	92 /100	:  0.2883620262145996 	26.787691593170166 	63.59376525878906
Batch:	93 /10

Batch:	11 /100	:  0.291184663772583 	3.508209228515625 	84.07793426513672
Batch:	12 /100	:  0.2812981605529785 	3.7905611991882324 	34.1467170715332
Batch:	13 /100	:  0.28768014907836914 	4.080246925354004 	34.78572463989258
Batch:	14 /100	:  0.2911524772644043 	4.372530221939087 	83.8360824584961
Batch:	15 /100	:  0.29094552993774414 	4.664862871170044 	137.85198974609375
Batch:	16 /100	:  0.28064870834350586 	4.947214365005493 	92.00542449951172
Batch:	17 /100	:  0.27951622009277344 	5.228087425231934 	107.07905578613281
Batch:	18 /100	:  0.2884335517883301 	5.517568349838257 	96.82917785644531
Batch:	19 /100	:  0.27945685386657715 	5.798168897628784 	125.61341094970703
Batch:	20 /100	:  0.2825779914855957 	6.0817625522613525 	30.940025329589844
Batch:	21 /100	:  0.2860379219055176 	6.36931037902832 	32.4068717956543
Batch:	22 /100	:  0.2854123115539551 	6.65591287612915 	82.45346069335938
Batch:	23 /100	:  0.2810394763946533 	6.938101530075073 	152.0740509033203
Batch:	24 /100	:  0.

Batch:	119 /100	:  0.29419374465942383 	34.635998249053955 	85.51302337646484
Batch:	120 /100	:  0.2881960868835449 	34.92573404312134 	77.2669906616211
Batch:	121 /100	:  0.30396437644958496 	35.23115944862366 	121.01502227783203
Batch:	122 /100	:  0.27478790283203125 	35.507895708084106 	80.04347229003906
Batch:	123 /100	:  0.2808094024658203 	35.78970956802368 	46.65058898925781
Batch:	124 /100	:  0.28383493423461914 	36.07486295700073 	71.5881576538086
Batch:	125 /100	:  0.2941861152648926 	36.37051558494568 	64.17329406738281
Batch:	126 /100	:  0.28370189666748047 	36.65557408332825 	62.124629974365234
Batch:	127 /100	:  0.2875862121582031 	36.94459319114685 	64.9560775756836
Batch:	128 /100	:  0.2742142677307129 	37.22104215621948 	70.76834869384766
Batch:	129 /100	:  0.28097081184387207 	37.50312852859497 	108.86109161376953
Batch:	130 /100	:  0.28462910652160645 	37.78896427154541 	27.093549728393555
Batch:	131 /100	:  0.28511548042297363 	38.075167655944824 	37.82993698120117


Batch:	48 /100	:  0.28295421600341797 	14.169642210006714 	77.67452239990234
Batch:	49 /100	:  0.27768397331237793 	14.448346614837646 	64.975830078125
Batch:	50 /100	:  0.2928147315979004 	14.742938995361328 	75.04146575927734
Batch:	51 /100	:  0.282900333404541 	15.027007579803467 	50.52314376831055
Batch:	52 /100	:  0.2815992832183838 	15.309986114501953 	68.63565826416016
Batch:	53 /100	:  0.2890942096710205 	15.600533723831177 	74.09783172607422
Batch:	54 /100	:  0.28086185455322266 	15.883001327514648 	70.86946868896484
Batch:	55 /100	:  0.27417898178100586 	16.15845489501953 	56.71770095825195
Batch:	56 /100	:  0.2859947681427002 	16.445712566375732 	77.19869232177734
Batch:	57 /100	:  0.28807663917541504 	16.735318899154663 	53.05147933959961
Batch:	58 /100	:  0.2858302593231201 	17.023595094680786 	92.30790710449219
Batch:	59 /100	:  0.2811143398284912 	17.306025981903076 	54.9898567199707
Batch:	60 /100	:  0.2811732292175293 	17.58821725845337 	64.54544830322266
Batch:	61 /10

In [ ]:
h[0][0].shape, h[0][1].shape

In [ ]:
plt.plot(train_loss)
plt.show()
plt.plot(valid_accuracy)
plt.show()
plt.plot(test_accuracy)
plt.show()
print(valid_accuracy)
print(test_accuracy)
print("validation accuracy is , ", max(valid_accuracy))
print("maximum test accuracy is , ", max(test_accuracy))
print("correct test accuracy i.e test accuracy where validation is highest is ", test_accuracy[valid_accuracy.index(max(valid_accuracy))])

In [ ]:
merity_model = merity.RNNModel(rnn_type='LSTM', 
                ntoken=238462,
                ninp=400,
                nhid=1150,
                nlayers=3,
                dropout=0.0,
                dropouth=0.5,
                dropouti=0.5,
                dropoute=0.5,
                wdrop=0,
                tie_weights=True)

pretrained_weights = torch.load('./ulmfit/wt103/fwd_wt103_enc.h5', map_location= lambda storage, loc: storage)

fastai_model = lm_rnn.RNN_Encoder(ntoken=238462, emb_sz=400, n_hid=1150, n_layers=3, pad_token=0,qrnn=False)

In [ ]:
from pprint import pprint
# print(pretrained_weights.keys())

# pprint(list(merity_model.state_dict().keys()))
# pprint(list(fastai_model.state_dict().keys()))


key_mapping = {
    'encoder.weight' : 'encoder.weight',
    'rnns.0.module.weight_ih_l0' : 'rnns.0.weight_ih_l0', 
    'rnns.0.module.bias_ih_l0' : 'rnns.0.bias_ih_l0', 
    'rnns.0.module.bias_hh_l0' : 'rnns.0.bias_hh_l0', 
    'rnns.0.module.weight_hh_l0_raw' : 'rnns.0.weight_hh_l0', 
    'rnns.1.module.weight_ih_l0' : 'rnns.1.weight_ih_l0', 
    'rnns.1.module.bias_ih_l0' : 'rnns.1.bias_ih_l0', 
    'rnns.1.module.bias_hh_l0': 'rnns.1.bias_hh_l0', 
    'rnns.1.module.weight_hh_l0_raw' : 'rnns.1.weight_hh_l0', 
    'rnns.2.module.weight_ih_l0' : 'rnns.2.weight_ih_l0', 
    'rnns.2.module.bias_ih_l0' : 'rnns.2.bias_ih_l0', 
    'rnns.2.module.bias_hh_l0': 'rnns.2.bias_hh_l0', 
    'rnns.2.module.weight_hh_l0_raw' : 'rnns.2.weight_hh_l0',
}

In [ ]:
# ew = pretrained_weights.pop('encoder.weight')
pretrained_weights.keys()

In [ ]:
pretrained_weights.pop('encoder_with_dropout.embed.weight')
pretrained_weights.keys()

In [ ]:
for k,v in key_mapping.items():
    pretrained_weights[v] = pretrained_weights.pop(k)

In [ ]:
merity_model, fastai_model

In [ ]:
[(k, v.shape) for k, v in merity_model.state_dict().items()]


In [ ]:
[(k, v.shape) for k, v in fastai_model.state_dict().items()]

In [ ]:
[(k,x.shape) for k, x in pretrained_weights.items()]

In [ ]:
merity_model.load_state_dict(pretrained_weights)